In [1]:
!ls

assignment_2.ipynb  brown.txt  exp_results.json  question.pdf


In [ ]:
from tqdm.auto import tqdm
from collections import defaultdict, OrderedDict
import numpy as np
from collections import Counter
import random, itertools
from sklearn.model_selection import KFold
import json
from concurrent.futures import ProcessPoolExecutor

### Dataset class

In [3]:
class txt_data():
    '''
    Master class to load  the dataset
    
    '''
    def __init__(self,path,split=0.9,shuffle=False):
        '''
        path: path to dataset
        split: train/val split
        shuffle: shuffle the dataset randomly before splitting or no.
        '''
        self.path = path
        
        # list of sentences of list of tuples defining pos_tag and words.
        self.data = self.___sent_to_tags___(self.___open___(self)) 
        if shuffle: random.shuffle(self.data)
        self.train =  self.data[:int(len(self.data)*split)]
        self.val = self.data[int(len(self.data)*split):]
        
        self.pos_tags = [i for i in set([j[0] for i in self.data for j in i])] #list of unique pos_tags
        self.vocab = [i for i in set([j[1] for i in self.data for j in i])] # list of vocab
            
    
    def ___open___(self,path):
        '''
        To read a text file
        path: path to the text file
        '''
        with open(self.path, "r") as f:
            return f.readlines()
    
    def ___sent_to_tags___(self,sents):
        '''
        converts a raw sentence to tags and words as a tuple
        '''
        dummy = []
        for i in tqdm(sents):
            d = []
            for j in i.split():
                try: d.append((j.split('_')[1],j.split('_')[0].lower()))
                except: pass #print(f"--{i}--{j}\n")
            dummy.append(d)
        return dummy

### Converitng emission/transition matrix to prob

In [4]:
def prob(matrix): 
    '''
    Converitng emission/transition matrix to prob
    matrix: list of lists format
    
    return: prob (2D numpy array)
    '''
    dummy = []
    for i in np.array(matrix):
        if np.sum(i) !=0: dummy.append(i/np.sum(i))
        else: dummy.append(i)
    return np.array(dummy)

### Smoothing

In [5]:
def smoothing(matrix, name='simple'):
    '''
    Smoothing operation to handle zero values
    matrix: 2D numpy array
    name: name of smoothing technique
    
    return: smoothed prob (numpy array)
    '''
    if name == 'simple':
        matrix[matrix == 0] = 0.000000001*matrix.mean()
        return matrix
    else: raise NotImplementedError

### emission matrix

In [6]:
def e_matrix(data,vocab,pos_tags,smooth='simple'):
    '''
    emission matrix: count of words given the pos_tag
    
    data: list of sents with the tags: format is list of list of sentences with tag and word as a tuple
    vocab: list of vocab
    pos_tags: list of pos_tags
    smooth: name of smooting technique to use
    
    return list of pos_tags and a list of matrix (pos_tags,vocab) with count(vocab/pos_tags)
    '''
    data = [i for j in data for i in j] # list of pos_tag and word pair: [('VB', 'suppose'), ('NP', 'lauren'),()]
    dummy = {s[0]:[] for s in data} # dictioanry with tag and list of words as key, value pair, {'AT' :['the', 'the', 'a', 'the'],}
    for i in data:
        dummy[i[0]].append(i[1])
        
    e_matrix = [] # emission count matrix with tag and count of words as row, column pair.
    for i in tqdm(pos_tags):
        if i in dummy.keys():
            count = Counter(dummy[i]) # count of words in a list. 
            e_matrix.append([count[j] for j in vocab]) # vocab(50k) loop takes time not the man loop
        else: e_matrix.append([0 for j in vocab])
    
    return pos_tags, smoothing(prob(e_matrix),name='simple')

### transition matrix

In [7]:
def t_matrix(data, pos_tags, n_gram=2, smooth='simple'):
    '''
    transition matrix: count of current_pos-tag given the last_pos-tag
    
    data = list of sents with the tags: format is list of list of sentences with tag and word as a tuple
    pos_tags = list of pos_tags
    n_gram = look back + 1 i.e, number of words tollok back to calculate the transition matrix
    smooth: name of smooting technique to use
    
    return a list of matrix (pos_tags**n_gram,pos_tags) with count(pos_tags/pos_tags**_gram)
    
    '''
    
    data = [[j[0]  for j in i] for i in data] # list of tags for each sentence: [['AT', 'NN', CS', '.'],[],] 
    #list of n_grams of tags, for 2 grams [['AT', 'NN'], ['NN', 'MD'], ['MD', 'HV'],[],]
    tags = [i[j:j+n_gram] for i in data for j in range(len(i) - n_gram + 1)]
    print(f" Number of {n_gram} grams present in the dataset: {len(tags)/1000000} million")
    # list of tags of cartesian product all the possible combination of pos_tag pair
    keys = [' '.join([j for j in i]) for i in list(itertools.product(pos_tags, repeat=n_gram-1))]
    keys.append("<s>") # start symbol <s>
     
    tags_ = {i:[] for i in keys} # initialize an empty dictionary, dict with
    for i in tags:
        a = ''
        for j in i[:-1]:
            a+= f"{j} "
        tags_[a.strip()].append(i[-1])
    tags_['<s>'] = [j for i in data for j in i] # list of all the tags in the dataset to count the start_prob
    
    t_matrix = [] # transition count matrix with n_gram tag and count of pos_tags as key, value pair.
    for i in tqdm(keys):
        count = Counter(tags_[i])
        d = [count[j] for j in pos_tags]
        t_matrix.append(d)

    return keys[:-1], smoothing(prob(t_matrix[:-1]),name='simple'), smoothing(prob([t_matrix[-1]])[0],name='simple') 

# Viterbi algorithm

In [8]:
def decoding(obs, pos_tags, t_tag, s_prob, t_prob, e_prob, n_gram, t_2=None):
    '''
    Viterbi decoding: "https://en.wikipedia.org/wiki/Viterbi_algorithm"
    
    obs: list of words in the sentence
    pos_tags: list of pos_tags
    s_prob: starting probabilty
    t_prob: transition probability
    e_prob: emission probability
    n_gram: number of words to look back +1
    
    return: top path
    '''
    path = { s:[] for s in pos_tags} # list of all the previous path pos_tags took
    curr_prob = {s:s_prob[s]*e_prob[s][obs[0]] for s in pos_tags} # first word/time-step prob

    for i in range(1, len(obs)):
        prev_prob = curr_prob
        curr_prob = {}
        for curr_state in pos_tags:
            if i > n_gram - 1 and n_gram > 2: # for the n_gram case
                max_prob, state = max(((prev_prob[last_state]*t_prob[f"{path[curr_state][-1:][0]} {last_state}"][curr_state]*e_prob[curr_state][obs[i]], last_state) 
                                           for last_state in pos_tags))
            elif n_gram > 2: # simple look back at the last word 
                max_prob, state = max(((prev_prob[last_state]*t_2[last_state][curr_state]*e_prob[curr_state][obs[i]], last_state) 
                                           for last_state in pos_tags))
            else:
                max_prob, state = max(((prev_prob[last_state]*t_prob[last_state][curr_state]*e_prob[curr_state][obs[i]], last_state) 
                                           for last_state in pos_tags))
            curr_prob[curr_state] = max_prob
            path[curr_state].append(state)

    # find the final largest probability
    max_prob = -1
    max_path = None
    for l, p in path.items():
        p.append(l)
        if curr_prob[l] > max_prob:
            max_path = p
            max_prob = curr_prob[l]
            
    return max_path



# Experimentation

In [9]:
path = "brown.txt"
brown = txt_data(path)
train, val = brown.train, brown.val
# pos_tags, vocab = brown.pos_tags, brown.vocab

In [10]:
n_splits = 3
cv = KFold(n_splits=n_splits, random_state=42, shuffle=True)
folds = {} # stores the data of train and test for n_split folds
for r, index in tqdm(enumerate(cv.split(brown.data)), total = n_splits):
    train = [j for k,j in enumerate(brown.data) if k in index[0]]
    test = [j for k,j in enumerate(brown.data) if k in index[1]]
    folds[f"fold_{r}"]= {
                            'train': train,
                            'test': test,
                            'pos_tags': brown.pos_tags,
                            'vocab': brown.vocab
    }

### parallel code implementation

In [11]:
# parallel code implementation
def run(decoding, txt):
    with ProcessPoolExecutor(max_workers=32) as executor:
        results = list( tqdm(executor.map(decoding, txt), total=len(txt)) )
    return results

## Training loop

In [ ]:
n_gram = 2 # markov assumption length: if it's 2 we need trigrams, thus value would be 3 (assumption length + 1) 
exp_results = {} # saving the results in the dict

print(f"Number of folds-/-n_gram used: {len(folds)}-/-{n_gram}\n\n")
for fold, data in tqdm(folds.items()):
    save_dict = {}
    print(f"\n\n{'-'*15}: {fold} \n\nlength of train and test data : {len(data['train'])}/{len(data['test'])}")
    
    save_dict['n_gram'] =  n_gram
    
    train = data['train']
    val = data['test']
    pos_tags = data['pos_tags']
    vocab = data['vocab']

    print(f"Creating the emission and transition probability matrix: ")
    e_tag, e_prob = e_matrix(train, vocab, pos_tags) # emission prob and pos_tags
    print(f"Finished emission prob matrix")
    # transition prob and tags
    if n_gram == 3: # if n_gram is 3 we also need 2_gram prob for the first index
        t_tag, t_prob_2, start_probability = t_matrix(train, pos_tags, 2) 
        t_tag, t_prob, start_probability = t_matrix(train, pos_tags, n_gram) 
    if n_gram == 2: t_tag, t_prob, start_probability = t_matrix(train, pos_tags, n_gram) 
    print(f"Finished tansition prob matrix")
    
    print(f"Creating emission and transition probability dict for O(1) searching")
    # creating emission and transition probability dict for O(1) searching
    if n_gram == 3: t_2 = {i: {pos_tags[f]:j for f,j in enumerate(t_prob_2[r])} for r,i in tqdm(enumerate(pos_tags), total=len(pos_tags))}
    else: t_2 = None
    transition_probability = {i: {pos_tags[f]:j for f,j in enumerate(t_prob[r])} for r,i in tqdm(enumerate(t_tag), total=len(t_tag))}
    emission_probability = {i: {vocab[f]:j for f,j in enumerate(e_prob[r])} for r,i in tqdm(enumerate(e_tag), total=len(e_tag))}
    start_prob = {i:start_probability[r] for r,i in enumerate(pos_tags)} 
    # start_prob = {i:1.0 for r,i in enumerate(t_tag)} # if wanted the same starting prob for all pos_tags
    
    print(f"Starting viterbi decoding: ")
    out = []
    true = []
#     print(f"Length of validation set is: {len(val)}")
#     for i in tqdm(val):
#         true.append(i)
#         obs = [j[1] for j in i]
#         out.append(decoding(obs, pos_tags, t_tag, start_prob, transition_probability, emission_probability, n_gram, t_2))
    
    txt = []
    for i in tqdm(val[:10]):
        true.append(i)
        obs = [j[1] for j in i]
        txt.append([obs, pos_tags, t_tag, start_prob, transition_probability, emission_probability, n_gram, t_2])
    
    print("starting the processes")
    temp = run(decoding, txt)
    print("saving the files")
    
    
    save_dict['true_val'] = true
    save_dict['pred_val'] = out
    
    print(f"starting metric calcualtion")
    true = [i[0] for j in true for i in j]
    out = [j for i in out for j in i]
    assert len(true) == len(out)

    
    #pos to index and index to pos dict for easy searching
    pos_index = {i:e for e,i in enumerate(pos_tags)}
    index_pos = {e:i for e,i in enumerate(pos_tags)}
    
    save_dict['pos_index'] = pos_index
    
    # adding the true and predictions in a single list
    expected = [pos_index[i] for i in  true]
    predicted = [pos_index[i] for i in  out]
    
    labels = sorted(pos_index.values())
    
    eps = 0

    dummy = OrderedDict()
    for i in labels:
        dummy[str(i)] = {str(j) : eps for j in labels}
    for r, i in enumerate(expected):
        dummy[str(i)][str(predicted[r])] +=1
    
    save_dict['conf_labels'] = [k for k in dummy.keys()]
    # confusion matrix
    conf_m = np.array([[j for j in i.values()] for i in dummy.values()]) # confusion matrix
    save_dict['conf_matrix'] =  conf_m.tolist()
    
    #classwise 
    pre = np.sum(conf_m,axis=0)
    rec = np.sum(conf_m,axis=1)
    precision = [i[r] for r,i in enumerate(conf_m)]/pre 
    recall = [i[r] for r,i in enumerate(conf_m)]/rec 
    save_dict['classwise_prec_and_rec'] = [precision.tolist(), recall.tolist()]
    
    # micro precision and recall
    precision = sum([i[r] for r,i in enumerate(conf_m)])/np.sum(pre) 
    recall = sum([i[r] for r,i in enumerate(conf_m)])/np.sum(rec)
    print("micro--",precision, recall)
    save_dict['micro_precision_and_recall'] = [precision.tolist(), recall.tolist()]
    
    # accuracy
    acc = len([i for i in range(len(expected)) if expected[i]==predicted[i]])/len(expected)
    print(f"Accuracy: {acc}")
    save_dict['accuracy'] = acc
    
    # micro f1 score
    f_1 = 2*((precision*recall)/(precision+recall))
    print(f"F1 score: {f_1}")
    save_dict['micro_f1'] = f_1
    
    exp_results[fold] = save_dict
    break

Number of folds-/-n_gram used: 3-/-2






---------------: fold_0 

length of train and test data : 36763/18382
Creating the emission and transition probability matrix: 



Finished emission prob matrix
 Number of 2 grams present in the dataset: 0.738769 million



Finished tansition prob matrix
Creating emission and transition probability dict for O(1) searching


In [ ]:
# saving results
with open("exp_results.json", "w")as f:
    json.dump(exp_results, f, indent=4)

In [ ]:
with open('exp_results.json', 'r') as f:
    exp_results = json.load(f)


In [ ]:
txt = [[(j[1], pos_tags, t_tag, start_prob, transition_probability, emission_probability) for j in i] for i in val]

def run(Viterbit, txt):
    with ProcessPoolExecutor(max_workers=32) as executor:
        results = list(tqdm((executor.map(Viterbit, txt)), total=len(txt)))
    return results

print("starting the processes")
temp = run(Viterbit, txt)
print("saving the files")